In [ ]:
import requests
import tempfile
import os
from serpapi import GoogleSearch

SERPAPI_KEY = 'edce5e465b9fcce369b8af59dc8ab33ed5fa09a76dfac924e392049ba7414de3'

def serpapi_search(image_url, max_results=5):
    """
    Perform a Google Reverse Image search via SerpAPI, using the URL.
    Automatically downloads the image if direct URL search fails (for Facebook/Instagram/etc.).
    
    Args:
        image_url (str): URL of the image to search.
        max_results (int): Number of top results to return.
    
    Returns:
        List[Dict]: Each dict contains key info from image_results.
    """
    from serpapi import GoogleSearch
    
    params = {
        "engine": "google_reverse_image",
        "image_url": image_url,
        "api_key": SERPAPI_KEY
    }
    
    try:
        # Try using the URL directly
        search = GoogleSearch(params)
        results = search.get_dict()
        if "image_results" in results and results["image_results"]:
            image_results = results["image_results"]
        else:
            raise Exception("No results from URL, may require download.")
    except Exception:
        # If URL fails, download image to a temporary file
        with tempfile.NamedTemporaryFile(delete=False, suffix=".jpg") as tmp_file:
            response = requests.get(image_url, timeout=10)
            response.raise_for_status()
            tmp_file.write(response.content)
            tmp_file_path = tmp_file.name
        
        # Use the downloaded file
        search = GoogleSearch({
            "engine": "google_reverse_image",
            "image_file": tmp_file_path,
            "api_key": SERPAPI_KEY
        })
        results = search.get_dict()
        image_results = results.get("image_results", [])
        
        # Clean up temporary file
        os.remove(tmp_file_path)

    # Extract only the authenticity-relevant fields
    output = []
    for res in image_results[:max_results]:
        output.append({
            "position": res.get("position"),
            "title": res.get("title"),
            "link": res.get("link"),
            "source": res.get("source"),
            "snippet": res.get("snippet"),
            "snippet_highlighted_words": res.get("snippet_highlighted_words")
        })
    
    return output



[]

In [34]:
import requests
import tempfile
import os
from serpapi import GoogleSearch

def serpapi_search(image_url, max_results=5):
    """
    Perform a Google Reverse Image search via SerpAPI, using the URL.
    Automatically downloads the image if direct URL search fails (for Facebook/Instagram/etc.).
    Returns structured results or a clear no-results object.
    
    Args:
        image_url (str): URL of the image to search.
        max_results (int): Number of top results to return.
    
    Returns:
        List[Dict] or Dict: Each dict contains key info from image_results,
                            or a dict with 'error' if nothing found.
    """
    params = {
        "engine": "google_reverse_image",
        "image_url": image_url,
        "api_key": SERPAPI_KEY
    }
    
    def extract_results(results):
        image_results = results.get("image_results", [])
        if not image_results:
            return {"error": "No results returned from SerpAPI for this query."}
        output = []
        for res in image_results[:max_results]:
            output.append({
                "position": res.get("position"),
                "title": res.get("title"),
                "link": res.get("link"),
                "source": res.get("source"),
                "snippet": res.get("snippet"),
                "snippet_highlighted_words": res.get("snippet_highlighted_words")
            })
        return output
    
    try:
        search = GoogleSearch(params)
        results = search.get_dict()
        output = extract_results(results)
        # If no results, try fallback download
        if isinstance(output, dict) and "error" in output:
            raise Exception("No results, attempting fallback download.")
        return output
    except Exception:
        # Download image to temporary file
        try:
            with tempfile.NamedTemporaryFile(delete=False, suffix=".jpg") as tmp_file:
                response = requests.get(image_url, timeout=10)
                response.raise_for_status()
                tmp_file.write(response.content)
                tmp_file_path = tmp_file.name
            
            # Search using local file
            search = GoogleSearch({
                "engine": "google_reverse_image",
                "image_file": tmp_file_path,
                "api_key": SERPAPI_KEY
            })
            results = search.get_dict()
            output = extract_results(results)
        except Exception as e:
            output = {"error": f"SerpAPI search failed: {str(e)}"}
        finally:
            # Clean up temporary file
            if 'tmp_file_path' in locals() and os.path.exists(tmp_file_path):
                os.remove(tmp_file_path)
        return output


In [35]:
serpapi_search('https://scontent.fsin4-1.fna.fbcdn.net/v/t51.82787-15/518238989_18043350404639957_3113069427818970275_n.jpg?_nc_cat=104&ccb=1-7&_nc_sid=127cfc&_nc_ohc=Ku7qZcm9RtQQ7kNvwFCzcJH&_nc_oc=AdmDCchBTwL8lNst4RLDgPhsKrGk7-iG-4neb5FIQ4WcsdaiR3JgUDMDktyc8PSOCCk&_nc_zt=23&_nc_ht=scontent.fsin4-1.fna&_nc_gid=qJfOyaaZHgK7nicGdAQ_5g&oh=00_AfXiI3du_zpGRcygbhPPmeXa5qIHsehh-Bv1d971G9D-FA&oe=68BB7E99')

{'error': 'No results returned from SerpAPI for this query.'}

In [36]:
from serpapi import GoogleSearch

params = {
  "engine": "google_reverse_image",
  "image_url": "https://businessdailymedia.com/images/joomgrabber/2023-05/119839f6be.jpeg",
  "api_key": "edce5e465b9fcce369b8af59dc8ab33ed5fa09a76dfac924e392049ba7414de3"
}

search = GoogleSearch(params)
results = search.get_dict()
print(results)

{'search_metadata': {'id': '68b59d34bde97944b47561c4', 'status': 'Success', 'json_endpoint': 'https://serpapi.com/searches/ac42cd5e1b5ca7c2/68b59d34bde97944b47561c4.json', 'created_at': '2025-09-01 13:18:44 UTC', 'processed_at': '2025-09-01 13:18:44 UTC', 'google_reverse_image_url': 'https://www.google.com/searchbyimage?image_url=https://businessdailymedia.com/images/joomgrabber/2023-05/119839f6be.jpeg&sbisrc=cr_1_5_2', 'raw_html_file': 'https://serpapi.com/searches/ac42cd5e1b5ca7c2/68b59d34bde97944b47561c4.html', 'total_time_taken': 6.1}, 'search_parameters': {'engine': 'google_reverse_image', 'image_url': 'https://businessdailymedia.com/images/joomgrabber/2023-05/119839f6be.jpeg', 'google_domain': 'google.com', 'device': 'desktop'}, 'search_information': {'query_displayed': 'language', 'total_results': 6, 'time_taken_displayed': 0.33, 'organic_results_state': 'Results for exact spelling'}, 'image_results': [{'position': 1, 'title': 'LanguageTool: Verificador gramatical por AI gratuit

In [37]:
print(results.keys())

dict_keys(['search_metadata', 'search_parameters', 'search_information', 'image_results'])


In [38]:
results['image_results']

[{'position': 1,
  'title': 'LanguageTool: Verificador gramatical por AI gratuito',
  'link': 'https://languagetool.org/pt-BR',
  'redirect_link': 'https://www.google.com/url?sa=t&source=web&rct=j&opi=89978449&url=https://languagetool.org/pt-BR&ved=2ahUKEwidl7iy07ePAxVmGbkGHQomHT8QFnoECAoQAQ&usg=AOvVaw2G1ZiNYzAGmyKncYo1H1yh',
  'displayed_link': 'https://languagetool.org › pt-BR',
  'favicon': 'https://serpapi.com/searches/68b59d34bde97944b47561c4/images/dce13cc2942f33357e5082512206fcf6cf36c7fa7f5b96b0.png',
  'snippet': 'O LanguageTool é um serviço gratuito de revisão para inglês, espanhol, alemão, francês e mais de 20 idiomas. Verifique instantaneamente se seu texto contém ...',
  'source': 'LanguageTool'},
 {'position': 2,
  'title': 'language - Tradução em português – Linguee',
  'link': 'https://www.linguee.com.br/ingles-portugues/traducao/language.html',
  'redirect_link': 'https://www.google.com/url?sa=t&source=web&rct=j&opi=89978449&url=https://www.linguee.com.br/ingles-portugu

In [27]:
image_results = results.get("image_results", [])

# Extract only the authenticity-relevant fields
output = []
for res in image_results[:3]:
    output.append({
        "position": res.get("position"),
        "title": res.get("title"),
        "link": res.get("link"),
        "source": res.get("source"),
        "snippet": res.get("snippet"),
        "snippet_highlighted_words": res.get("snippet_highlighted_words")
    })
print(output)

[{'position': 1, 'title': 'Money Mules - FBI', 'link': 'https://www.fbi.gov/how-we-can-help-you/scams-and-safety/common-frauds-and-scams/money-mules', 'source': 'FBI (.gov)', 'snippet': 'A money mule is someone who transfers or moves illegally acquired money on behalf of someone else. Criminals recruit money mules to help launder proceeds ...', 'snippet_highlighted_words': ['money mule']}, {'position': 2, 'title': 'Money mule - Wikipedia', 'link': 'https://en.wikipedia.org/wiki/Money_mule', 'source': 'Wikipedia', 'snippet': 'Money mule ... A money mule, sometimes called a "smurfer", is a person who transfers money acquired illegally, such as by theft or fraud. Money mules transfer ...', 'snippet_highlighted_words': ['Money mule', 'money mule']}, {'position': 3, 'title': '27 to be charged over money mule activities in scam cases ...', 'link': 'https://sg.news.yahoo.com/27-charged-over-money-mule-091000026.html', 'source': 'Yahoo News Singapore', 'snippet': 'SINGAPORE - A total of 27 peo

In [ ]:
from serpapi import GoogleSearch

def get_image_authenticity_info(image_url, api_key, max_results=5):
    """
    Perform a Google Reverse Image search via SerpAPI and return
    only the most relevant information for determining authenticity.
    
    Args:
        image_url (str): URL of the image to search.
        api_key (str): Your SerpAPI key.
        max_results (int): Number of top results to return.
    
    Returns:
        List[Dict]: Each dict contains the key info from image_results.
    """
    params = {
        "engine": "google_reverse_image",
        "image_url": image_url,
        "api_key": api_key
    }

    search = GoogleSearch(params)
    results = search.get_dict()

    # Get the main image results
    image_results = results.get("image_results", [])

    # Extract only the authenticity-relevant fields
    output = []
    for res in image_results[:max_results]:
        output.append({
            "position": res.get("position"),
            "title": res.get("title"),
            "link": res.get("link"),
            "source": res.get("source"),
            "snippet": res.get("snippet"),
            "snippet_highlighted_words": res.get("snippet_highlighted_words")
        })

    return output


Results for https://cassette.sphdigital.com.sg/image/straitsti…5b6da044735695b786f50a23c715de78605a5b6375e?w=900:
-----
Results for https://cassette.sphdigital.com.sg/image/straitsti…5f98e1b7bfa74492896fedf3310276d1d55e76a8092?w=720:
-----
Results for https://cassette.sphdigital.com.sg/image/straitsti…73408623c44b6748cf13d48cd804fa286cc49089544?w=900:
-----


In [ ]:
# agent.py
from strands import Agent
from strands.llms import Anthropic
from tools import serpapi_search, sensity_check
import os

# Set up the LLM (GPT-4o-mini or other)
llm = OpenAI(api_key="YOUR_OPENAI_API_KEY", model="gpt-4o-mini")

# Create agent and register tools
agent = Agent(llm=llm)
agent.add_tool(serpapi_search)
agent.add_tool(sensity_check)

# Prompt template instructing the LLM how to act
AGENT_PROMPT = """
You are an AI image verification assistant. 
For each image URL:
1. Decide which tool(s) to call (SerpAPI for reverse image search, Sensity for deepfake detection).
2. Call the chosen tool(s) and collect results.
3. Return a JSON with:
    - image_url
    - tools_called (list)
    - assessment ("likely_real", "possibly_fake", "fake")
    - tool_results (output from each tool)
"""

def run_agent_on_images(image_urls, serpapi_key, sensity_key=None):
    """
    Runs the Strands LLM agent on one or more image URLs.
    """
    if isinstance(image_urls, str):
        image_urls = [image_urls]

    results = []

    for url in image_urls:
        # Instruct LLM to reason, call tools, and produce assessment
        response = agent.run(
            prompt=AGENT_PROMPT + f"\nImage URL: {url}",
            input_variables={
                "image_url": url,
                "serpapi_key": serpapi_key,
                "sensity_key": sensity_key
            }
        )
        results.append(response)

    return results


NameError: name 'output' is not defined

In [ ]:
# agent_claude.py
from strands import Agent
from strands.llms import Anthropic
from tools import serpapi_search, sensity_check
import os

# Set your Anthropic API key in environment
# export ANTHROPIC_API_KEY="your_api_key"  (Linux/Mac)
# setx ANTHROPIC_API_KEY "your_api_key"   (Windows)

api_key = os.environ.get("ANTHROPIC_API_KEY")

# Initialize Claude 3 Haiku
llm = Anthropic(api_key=api_key, model="claude-3-haiku")

# Create agent
agent = Agent(llm=llm)

# Register tools
agent.add_tool(serpapi_search)
agent.add_tool(sensity_check)

# Prompt template
AGENT_PROMPT = """
You are an AI image verification assistant using Claude 3 Haiku. 
For each image URL:
1. Decide which tool(s) to call (SerpAPI for reverse image search, Sensity for deepfake detection).
2. Call the chosen tool(s) and collect results.
3. Return a JSON with:
    - image_url
    - tools_called (list)
    - assessment ("likely_real", "possibly_fake", "fake")
    - tool_results (output from each tool)
"""

def run_agent_on_images(image_urls, serpapi_key, sensity_key=None):
    if isinstance(image_urls, str):
        image_urls = [image_urls]

    results = []

    for url in image_urls:
        response = agent.run(
            prompt=AGENT_PROMPT + f"\nImage URL: {url}",
            input_variables={
                "image_url": url,
                "serpapi_key": serpapi_key,
                "sensity_key": sensity_key
            }
        )
        results.append(response)

    return results

# ---------------- Example Usage ----------------
if __name__ == "__main__":
    SERPAPI_KEY = "YOUR_SERPAPI_KEY"
    SENSITY_KEY = None

    test_images = [
        "https://i.imgur.com/5bGzZi7.jpg",
        "https://i.imgur.com/abc123.jpg"
    ]

    agent_results = run_agent_on_images(test_images, SERPAPI_KEY, SENSITY_KEY)

    for r in agent_results:
        print(r)
        print("-----")


In [ ]:
import boto3
import os
from tools import serpapi_search, sensity_check

# Initialize Bedrock client
bedrock_client = boto3.client("bedrock-runtime", region_name="us-east-1")

def ask_claude_bedrock(prompt, system="You are an image verification assistant."):
    """Send a prompt to Claude via AWS Bedrock"""
    response = bedrock_client.converse(
        modelId="us.anthropic.claude-3-haiku-20240307-v1:0",  # Haiku model
        messages=[{"role": "user", "content": [{"text": prompt}]}],
        system=[{"text": system}],
        inferenceConfig={"temperature": 0, "maxTokens": 200}
    )
    return response["output"]["message"]["content"][0]["text"]

# ---------------- AGENT ----------------
AGENT_PROMPT = """
You are an AI image verification assistant.
You have access to two tools:
- SerpAPI (reverse image search)
- Sensity (deepfake detection)

For each image:
1. Decide which tools to use.
2. Collect tool outputs.
3. Return a JSON with:
   {
     "image_url": "...",
     "tools_called": [...],
     "assessment": "likely_real / possibly_fake / fake",
     "tool_results": {...}
   }
"""

def run_agent_on_images(image_urls, serpapi_key, sensity_key=None):
    if isinstance(image_urls, str):
        image_urls = [image_urls]

    results = []

    for url in image_urls:
        # Run SerpAPI by default (Sensity optional)
        serp_results = serpapi_search(url, serpapi_key)
        sensity_results = None
        if sensity_key:
            sensity_results = sensity_check(url, sensity_key)

        # Construct tool context for Claude
        tool_context = {
            "serpapi": serp_results,
            "sensity": sensity_results
        }

        prompt = (
            AGENT_PROMPT
            + f"\nImage URL: {url}\nTool outputs:\n{tool_context}\n"
            + "Now decide authenticity."
        )

        # Ask Claude through Bedrock
        decision = ask_claude_bedrock(prompt)
        results.append(decision)

    return results

# ---------------- Example Usage ----------------
if __name__ == "__main__":
    SERPAPI_KEY = "YOUR_SERPAPI_KEY"
    SENSITY_KEY = None  # if you don’t have it yet

    test_images = [
        "https://i.imgur.com/5bGzZi7.jpg",
        "https://i.imgur.com/abc123.jpg"
    ]

    output = run_agent_on_images(test_images, SERPAPI_KEY, SENSITY_KEY)
    for r in output:
        print(r)
        print("-----")


In [41]:
def serpapi_search(image_url, max_results=5):
    """
    Perform a Google Reverse Image search via SerpAPI and return
    only the most relevant information for determining authenticity.
    
    Args:
        image_url (str): URL of the image to search.
        api_key (str): Your SerpAPI key.
        max_results (int): Number of top results to return.
    
    Returns:
        List[Dict]: Each dict contains the key info from image_results.
    """
    params = {
        "engine": "google_reverse_image",
        "image_url": image_url,
        "api_key": 'edce5e465b9fcce369b8af59dc8ab33ed5fa09a76dfac924e392049ba7414de3'
    }

    search = GoogleSearch(params)
    results = search.get_dict()

    # Get the main image results
    image_results = results.get("image_results", [])

    # Extract only the authenticity-relevant fields
    output = []
    for res in image_results[:max_results]:
        output.append({
            "position": res.get("position"),
            "title": res.get("title"),
            "link": res.get("link"),
            "source": res.get("source"),
            "snippet": res.get("snippet"),
            "snippet_highlighted_words": res.get("snippet_highlighted_words")
        })

    return output

serpapi_search('https://i.imgur.com/5bGzZi7.jpg')

[{'position': 1,
  'title': 'Elon Musk - Wikipedia',
  'link': 'https://en.wikipedia.org/wiki/Elon_Musk',
  'source': 'Wikipedia',
  'snippet': 'Born to a wealthy family in Pretoria, South Africa, Musk emigrated in 1989 to Canada; he had obtained Canadian citizenship at birth through his Canadian-born ...',
  'snippet_highlighted_words': ['Musk']},
 {'position': 2,
  'title': 'Elon Musk (@elonmusk) / X',
  'link': 'https://x.com/elonmusk',
  'source': 'X · elonmusk',
  'snippet': "Elon is right. The British government is literally cutting seven-year checks to hotel chains to warehouse illegals. Seven years. That's not a “temporary crisis ...",
  'snippet_highlighted_words': ['Elon']},
 {'position': 3,
  'title': "Elon Musk says he will support Florida's DeSantis if he runs ...",
  'link': 'https://www.nbcnews.com/politics/politics-news/elon-musk-ron-desantis-president-election-trump-rcna58804',
  'source': 'NBC News',
  'snippet': 'Musk had previously said in June he was leaning toward

In [44]:
import json
def serpapi_search(image_url, max_results=5):
    """
    Perform a Google Reverse Image search via SerpAPI and return
    only the most relevant information for determining authenticity.
    
    Args:
        image_url (str): URL of the image to search.
        api_key (str): Your SerpAPI key.
        max_results (int): Number of top results to return.
    
    Returns:
        List[Dict]: Each dict contains the key info from image_results.
    """
    params = {
        "engine": "google_reverse_image",
        "image_url": image_url,
        "api_key": SERPAPI_KEY
    }

    search = GoogleSearch(params)
    results = search.get_dict()

    # Get the main image results
    image_results = results.get("image_results", [])

    # Extract only the authenticity-relevant fields
    output = []
    for res in image_results[:max_results]:
        output.append({
            "position": res.get("position"),
            "title": res.get("title"),
            "link": res.get("link"),
            "source": res.get("source"),
            "snippet": res.get("snippet"),
            "snippet_highlighted_words": res.get("snippet_highlighted_words")
        })

    return json.dumps(output)

serpapi_search('https://cassette.sphdigital.com.sg/image/straitstimes/756f80527c8d331fa93c409cd80b7c5fd911c8c8cda2025f350e1fa8d35f201a?w=900')

'[{"position": 1, "title": "Statement on Misinformation Circulating on Social Media", "link": "https://www.sps.gov.sg/statement-on-misinformation-circulating-on-social-media/", "source": "Singapore Prison Service", "snippet": "The Singapore Prison Service (SPS) is aware of several social media posts circulating online that Singapore has launched the world\'s first floating prison.", "snippet_highlighted_words": ["Singapore Prison", "Singapore"]}, {"position": 2, "title": "Singapore Prison Service debunks online claims that it ...", "link": "https://www.straitstimes.com/singapore/singapore-prison-service-debunks-online-claims-that-it-launched-the-worlds-first-floating-prison", "source": "The Straits Times", "snippet": "Singapore Prison Service debunks online claims that it launched \'the world\'s first floating prison\' ... Social media posts ...", "snippet_highlighted_words": ["Singapore Prison", "floating prison"]}, {"position": 3, "title": "Baseless Singapore \'floating prison\' clai

In [43]:
serpapi_search('https://businessdailymedia.com/images/joomgrabber/2023-05/119839f6be.jpeg')

[{'position': 1,
  'title': 'LanguageTool: Verificador gramatical por AI gratuito',
  'link': 'https://languagetool.org/pt-BR',
  'source': 'LanguageTool',
  'snippet': 'O LanguageTool é um serviço gratuito de revisão para inglês, espanhol, alemão, francês e mais de 20 idiomas. Verifique instantaneamente se seu texto contém ...',
  'snippet_highlighted_words': None},
 {'position': 2,
  'title': 'language - Tradução em português – Linguee',
  'link': 'https://www.linguee.com.br/ingles-portugues/traducao/language.html',
  'source': 'Linguee',
  'snippet': 'language substantivo (plural: languages)— · língua f (frequentemente utilizado) (plural: línguas f) · idioma m · linguagem f. The man explained his plan in ...',
  'snippet_highlighted_words': ['language']},
 {'position': 3,
  'title': 'Significado de language no Essential English Dictionary',
  'link': 'https://dictionary.cambridge.org/pt/dicionario/essential-british-english/language',
  'source': 'Cambridge Dictionary',
  'snippet': 